In [245]:
from pymongo import MongoClient
import pandas as pd
import sys

In [227]:
client = MongoClient()

### In our database "crunchbase": we have the companies collection & we create one for offices

The requirements for the new location is that near companies are related to both: tech and design, as well as being founded after the year 2000

In [4]:
db = client.crunchbase
companies = db.companies
offices = db.offices


In [5]:
unwind = {'$unwind': {'path': '$offices'}}
dropnull = {'$match':{'offices.latitude':{'$ne': None}, 'offices.longitude':{'$ne': None}}}
geo_code = {'$set': {'geo_coord': {'type': 'Point', 'coordinates': ['$offices.longitude', '$offices.latitude']}}}
remove = {'$project':{'_id':0}}

filt_design = {"$match": { "$and": [{"tag_list" : { "$regex" : ".*[Tt]ech.*" }},{"tag_list" : { "$regex" : ".*[Dd]esign.*" }}, {"founded_year":{"$gte":2000}}]}}


In [6]:
filtered_pipeline = [ unwind, dropnull, geo_code, remove, filt_design]

In [7]:
result = companies.aggregate(filtered_pipeline)

In [8]:
result

In [ ]:
 offices.insert_many(result)

In [205]:
offices.create_index([("geo_coord","2dsphere")])

'geo_coord_2dsphere'

In [218]:
countries = offices.find({},{ "offices.state_code":1, "_id":0} )

In [219]:
 countries = list(countries)

In [220]:
states = pd.DataFrame(countries)

In [250]:
states.head()

,offices
0,{'state_code': 'CA'}
1,{'state_code': 'IL'}
2,{'state_code': None}
3,{'state_code': 'CO'}
4,{'state_code': 'NY'}


In [196]:
def dropnest(elemento):
    '''This takes a dictionary and flattens it
    Takes: dictionary
    Returns: nested key'''
    for i in elemento.values():
        for k2 in i.values():
            return k2      

In [222]:
estados = list(map(dropnest, countries))
estados = pd.DataFrame(estados)

In [223]:
estados.columns = ['American States']

In [228]:
estados.head()

,American States
0,CA
1,IL
2,None
3,CO
4,NY


In [149]:
dropingnone = estados[estados['American States'].map(lambda x: str(x)!="None")]

In [249]:
estados.value_counts()

American States
CA                 18
NY                  6
RI                  4
CO                  2
FL                  2
GA                  2
IL                  2
NJ                  2
TX                  2
WY                  2
dtype: int64

**California is the way to go:**

In [231]:
cali = offices.find({"offices.state_code":{ "$regex" : ".*CA.*" }}, {"geo_coord":1, "_id":0}).sort("geo_coord", 1)

In [232]:
list(cali)[:5]

[{'geo_coord': {'type': 'Point', 'coordinates': [-122.4264417, 37.7550372]}},
 {'geo_coord': {'type': 'Point', 'coordinates': [-122.4264417, 37.7550372]}},
 {'geo_coord': {'type': 'Point', 'coordinates': [-122.421242, 37.778687]}},
 {'geo_coord': {'type': 'Point', 'coordinates': [-122.421242, 37.778687]}},
 {'geo_coord': {'type': 'Point', 'coordinates': [-122.421242, 37.778687]}}]

### Lets order them by nearest locations:

In [83]:
maxpoint = {
        "type":"Point",
        "coordinates":[-122.4264417, 37.7550372] 
    }

In [84]:
def dist(pointA, pointB=(0,0)):
    dist = 0
    for i in range(len(pointA)):
        dist += (pointA[i] - pointB[i])**2
    return (dist **.5)*111000

In [85]:
dist([-122.4264417, 37.7550372],[-118.439532, 33.988687]) 

608790.6569206794

#### There´s a difference of 60 km. Which we reduce to 10:

In [256]:
res = pd.DataFrame(offices.find({"geo_coord":{"$near":maxpoint,"$maxDistance":10000}}, {"geo_coord.coordinates":1,"_id":0, "name":1}))


In [258]:
res.head()

,name,geo_coord
0,Netbiscuits,"{'coordinates': [-122.4264417, 37.7550372]}"
1,Netbiscuits,"{'coordinates': [-122.4264417, 37.7550372]}"
2,Popego,"{'coordinates': [-122.421242, 37.778687]}"
3,Popego,"{'coordinates': [-122.421242, 37.778687]}"
4,Popego,"{'coordinates': [-122.421242, 37.778687]}"


In [253]:
new_res = res.geo_coord.apply(pd.Series)

In [180]:
lat_long = pd.DataFrame(new_res["coordinates"].to_list(), columns=['longitude', 'latitude'])

In [189]:
ofis = pd.merge(res, lat_long, left_index=True, right_index=True)

In [246]:
ofis.head()

,name,longitude,latitude
0,Netbiscuits,-122.426442,37.755037
2,Popego,-122.421242,37.778687
6,Ecolect,-122.410111,37.776769
10,Digg,-122.394523,37.764726


### Final clean of the mongo data:

In [240]:
del ofis["geo_coord"]

In [242]:
ofis.drop_duplicates(inplace=True)

In [244]:
ofis.to_csv("mongo_search.csv")